# Building Kubeflow Pipeline

In [ ]:
#Installer Kubeflow
!pip install kfp

In [ ]:
import kfp
import kfp.components as comp

In [ ]:
# Creer un compte dans Docker
!docker login -u oumaimafariss -p demo1234

In [ ]:
#créer image Docker 
!docker build -t ouamimafariss/target .

In [ ]:
#Déployer l'image Docker 
!docker push ouamimafariss/target

## Mise en place du pipeline Data


In [ ]:
# creation de component pour chaque etape du pipeline
@kfp.dsl.component
def etape1_data_pipeline():
    # Defining component configuration
    Etape1 = kfp.dsl.ContainerOp(
        name='Clinical_trials_table',
        image='ouamimafariss/target',
        command=['python', 'Etape1_Clinical_trials.py'],
        )
    return Etape1

In [ ]:
# creation de component pour chaque etape du pipeline
@kfp.dsl.component
def etape2_data_pipeline():    
    # Defining component configuration
    Etape2 = kfp.dsl.ContainerOp(
        name='pubMed_table',
        image='ouamimafariss/target',
        command=['python', 'Etape2_PubMed.py'],
        )
    return Etape2


In [ ]:
# creation de component pour chaque etape du pipeline
@kfp.dsl.component
def etape3_data_pipeline():    
    # Defining component configuration
    Etape3 = kfp.dsl.ContainerOp(
        name='Journal_table',
        image='ouamimafariss/target',
        command=['python', 'Etape3_Journal.py'],
        )
    return Etape3

In [ ]:
# creation de component pour chaque etape du pipeline
@kfp.dsl.component
def etape4_data_pipeline():
    # Defining component configuration
    Etape_final = kfp.dsl.ContainerOp(
        name='table_finale',
        image='ouamimafariss/target',
        command=['python', 'Etape4_final_table.py'],
        )
    
    return Etape_final

## Final Kubeflow pipeline Definition

In [ ]:
# Pipeline final
@kfp.dsl.pipeline(
  name="Kubeflow ML pipeline on drugs",
  description="Kubeflow ML pipeline on drugs"
)
def pipeline():
    pipeline_1 = etape1_data_pipeline()
    pipeline_1.execution_options.caching_strategy.max_cache_staleness = "P0D"
    pipeline_2 = etape2_data_pipeline().after(pipeline_1)
    pipeline_2.execution_options.caching_strategy.max_cache_staleness = "P0D"
    pipeline_3 = etape3_data_pipeline().after(pipeline_2)
    pipeline_3.execution_options.caching_strategy.max_cache_staleness = "P0D"
    pipeline_4 = etape3_data_pipeline().after(pipeline_3)
    pipeline_4.execution_options.caching_strategy.max_cache_staleness = "P0D"

## Connecte to deplyed Kubeflow pipeline Endpoint (GCP)

In [ ]:
# Create kfp client
client = kfp.Client(host='lien du gcp du client')